In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
data_dir = "/kaggle/input/microscopic-peripheral-blood-cell-images/PBC_dataset_normal_DIB"

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])
indices = list(range(len(full_dataset)))

In [3]:
dataper = [0.01, 0.025, 0.05, 0.075, 0.1, 0.2, 0.3]
accuracies = {
    "test" : []
}
test_acc_per = 0

# Ensure that the test set is fixed with 4000 images
train_idx, test_idx = train_test_split(indices, test_size=4000, random_state=42)
test_dataset = Subset(full_dataset, test_idx)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Test set: {len(test_dataset)} images")

Test set: 4000 images


In [4]:
mod_dataper = [i * (len(full_dataset)/(len(full_dataset)-4000)) for i in dataper]
mod_dataper

[0.013055300947143293,
 0.032638252367858236,
 0.06527650473571647,
 0.0979147571035747,
 0.13055300947143295,
 0.2611060189428659,
 0.3916590284142988]

In [5]:
for i,j in zip(mod_dataper,dataper):
    # Split the remaining indices for training and validation
    remaining_indices = [idx for idx in indices if idx not in test_idx]
    train_idx, val_idx = train_test_split(remaining_indices, train_size=i, random_state=42)
    val_idx, val_idx_no_use = train_test_split(val_idx, train_size=0.02, random_state=42)

    train_dataset = Subset(full_dataset, train_idx)
    val_dataset = Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    print(f"Train set: {len(train_dataset)} images")
    print(f"Validation set: {len(val_dataset)} images")

    num_classes = 8

    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 8)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
        best_acc = 0.0
        for epoch in range(num_epochs):
            print(f'Epoch {epoch+1}/{num_epochs}')
            print('-' * 10)

            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_acc = correct.double() / len(train_loader.dataset)

            print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Validation phase
            model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * inputs.size(0)
                    _, preds = torch.max(outputs, 1)
                    correct += torch.sum(preds == labels.data)
                    total += labels.size(0)

            val_loss = val_loss / len(val_loader.dataset)
            val_acc = correct.double() / len(val_loader.dataset)

            print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

            # Step scheduler
            scheduler.step()

            # Save the model if validation accuracy improves
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), f'best_resnet50_{j*100}.pth')
                print(f"Best model saved with accuracy: {best_acc:.4f}")
                train_acc_per = round(epoch_acc.item(), 4)
                val_acc_per = round(val_acc.item(), 4)

    # Train the model with dynamic number of epochs
    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

    # Evaluate the model on the test dataset
    def test_model(model, test_loader):
        model.load_state_dict(torch.load(f'best_resnet50_{j*100}.pth'))
        model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)

        test_acc = correct.double() / total
        print(f'Test Accuracy: {test_acc:.4f}')
        return round(test_acc.item(), 4)

    # Test the model
    test_acc_per = test_model(model, test_loader)
    accuracies["test"].append(test_acc_per)
    
    print(f'Done for {j*100} % data')


Train set: 170 images
Validation set: 258 images


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

 12%|█▏        | 11.9M/97.8M [00:00<00:00, 125MB/s]

...

Val Loss: 0.3097 Acc: 0.8805


Best model saved with accuracy: 0.8805
Epoch 6/10
----------


Train Loss: 0.3227 Acc: 0.8982


Val Loss: 0.4985 Acc: 0.8428
Epoch 7/10
----------


Train Loss: 0.2613 Acc: 0.9148


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
data = {
        'Data Split': [f'{i*100}%' for i in dataper],
        'Classifier': ['ResNET50 Baseline' for i in range(7)],
        'Test Acc (%)': [round(accuracies['test'][i]*100,2) for i in range(7)]
}

df = pd.DataFrame(data)
fig, ax = plt.subplots(figsize=(8, 2))
ax.axis('tight')
ax.axis('off')
tbl = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1.2, 1.2) 
plt.title(f'ResNET50 Comparison Table', pad=20)
plt.show()